# Validation BraTS


In [ ]:
! pip install elasticdeform
! pip install tensorflow_addons
! pip install gdown

In [ ]:
import tensorflow as tf
import numpy as np
import tarfile
import nibabel as nib
import glob
import time
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from sys import stdout
import matplotlib.pyplot as plt
import matplotlib.image as mpim
from scipy.ndimage.interpolation import affine_transform
from sklearn.model_selection import train_test_split
import json

import numpy as np
from scipy.ndimage.interpolation import affine_transform
import elasticdeform
import multiprocessing as mp
import os
import numpy as np
import tensorflow as tf
import nibabel as nib
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Conv3D, Conv3DTranspose, Dropout, ReLU, LeakyReLU, Concatenate, ZeroPadding3D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
import tensorflow_addons as tfa
from tensorflow_addons.layers import InstanceNormalization
from tensorflow.keras.layers import BatchNormalization
import numpy as np
from scipy import ndimage

from sklearn.metrics import precision_score
#Importing tqdm function of tqdm module 
from tqdm import tqdm  
from time import sleep 
import gdown

In [ ]:

inputs_file = 'https://drive.google.com/file/d/1-1DvEsuUJlakF6PCFH_Fd9BWZElXZnLl/view?usp=sharing'
inputs_url = 'https://drive.google.com/uc?id=1-1DvEsuUJlakF6PCFH_Fd9BWZElXZnLl'


inputs_output = 'inputs.zip'

gdown.download(inputs_url, inputs_output, quiet=False)

In [ ]:
! unzip ./inputs.zip -d ./INPUTS
! mv -v ./INPUTS/content/drive/MyDrive/GANs/BrainTumorDetection/INPUTS_EVAL/* ./INPUTS
! rm -rf ./INPUTS/content
! rm ./inputs.zip

In [ ]:
EXEC = 1
kze = 4
VERSION = 'eval_brats21'

INPUT_PATH = './INPUTS'

PATH = './RESULTS' 

DATA_PATH = '../input/brats-valid'
PATHVAL='./RESULTS/VALID'

In [ ]:
if EXEC!=1:
    results_file = 'https://drive.google.com/file/d/10jd2xXbL5SbADb0aco2KSYgR5GCuCU_I/view?usp=sharing'
    results_url = 'https://drive.google.com/uc?id=10jd2xXbL5SbADb0aco2KSYgR5GCuCU_I'

    results_output = 'results.zip'

    gdown.download(results_url, results_output, quiet=False)
    
else:
    ! mkdir ./RESULTS
    ! mkdir ./RESULTS/VALID

In [ ]:
if EXEC!=1:
    ! unzip ./results.zip -d ./RESULTS
    ! mv -v ./RESULTS/content/drive/MyDrive/GANs/BrainTumorDetection/RESULTS_KAGGLE/VERSION_$VERSION/RESULTS_{EXEC-1}/* ./RESULTS
    ! rm -rf ./RESULTS/content
    ! rm ./results.zip  

In [ ]:
#load les imags et appliquer les augmentation

def load_img(img_files):
    ''' Load one image and its target form file
    '''
    N = len(img_files)
    # target
#     y = nib.load(img_files[N-1]).get_fdata(dtype='float32', caching='unchanged')
#     y = y[40:200,34:226,8:136]
#     y[y==4]=3
    
    X_norm = np.empty((240, 240, 155, 4))
    for channel in range(N):
        X = nib.load(img_files[channel]).get_fdata(dtype='float32', caching='unchanged')
        XX = nib.load(img_files[channel])
        img_affine=XX.affine
        brain = X[X!=0] 
        brain_norm = np.zeros_like(X) # background at -100
        norm = (brain - np.mean(brain))/np.std(brain)#normalisation du brain
        brain_norm[X!=0] = norm
        X_norm[:,:,:,channel] = brain_norm        
        
    X_norm = X_norm[40:200,34:226,8:136,:]    
    del(X, brain, brain_norm)
    
    return X_norm,img_affine
    
    
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size=4, dim=(160,192,128), n_channels=4, n_classes=4, shuffle=True, augmentation=False, patch_size=128, n_patches=1):
        'Initialization'
        self.list_IDs = list_IDs
        self.batch_size = batch_size
        self.dim = dim
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.augmentation = augmentation
        self.patch_size = patch_size
        self.n_patches = n_patches
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return len(self.list_IDs) // self.batch_size

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data     
        X,Z,AFF= self.__data_generation(list_IDs_temp)
#         if self.augmentation == True:
#             X, y = self.__data_augmentation(X, y)
        
        if index == self.__len__()-1:
            self.on_epoch_end()
        
        return X,Z,AFF

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
  
    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        Z=[]
        AFF=[]
        # Generate data
        for i, IDs in enumerate(list_IDs_temp):
            # Store sample
            X[i],affine = load_img(IDs)
            AFF.append(affine)
            Z.append(IDs[0])

        return X.astype('float32'),Z,AFF



In [ ]:
#MODELS Unet3DGan

def double_conv(x, Nf, ks, norm=True):
      for ss in range(2):
          x = Conv3D(Nf, kernel_size=ks, strides=1, kernel_initializer='he_normal', padding='same')(x)
          #a chaque etape effectuer la convolution afin d obtenir shape/2 avec nombre de filtre comme 4 eme D
          if (norm):
              x = BatchNormalization()(x) #Normaliser l image seulement 
          x = ReLU()(x) #Activer la sortie
      return x   

def Generator():
    '''
    Generator model
    '''
    
    def encoder_step(layer, Nf, ks, norm=True):
        x = double_conv(layer, Nf/2, ks, norm)
        x = Conv3D(Nf, kernel_size=ks, strides=2, kernel_initializer='he_normal', padding='same')(x)  
#         x = Dropout(0.1)(x) #met a zero 0.2 des donnees afin d eviter l overfiting

        return x

    def bottlenek(layer, Nf, ks):
        x = Conv3D(Nf, kernel_size=ks, strides=2, kernel_initializer='he_normal', padding='same')(layer)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        #chaque etape de convolution est concatene avec la precedente (ici prblm de size)
        for i in range(4):
            y = Conv3D(Nf, kernel_size=ks, strides=1, kernel_initializer='he_normal', padding='same')(x)
            x = BatchNormalization()(y)
            x = ReLU()(x)
            x = Concatenate()([x, y])
#             print(x.shape)
        return x

    def decoder_step(layer, layer_to_concatenate, Nf, ks):
        
#         x = Dropout(0.1)(layer)
        x=layer
        x = Conv3DTranspose(Nf, kernel_size=ks, strides=2, padding='same', kernel_initializer='he_normal')(x)
        x = Concatenate()([x, layer_to_concatenate])
        x = double_conv(x, Nf, ks)
        return x

    layers_to_concatenate = []
    inputs = Input((128,128,128,4), name='input_image')
    Nfilter_start = 24
    depth = 5
    ks = 4
    x = inputs

    # encoder
    for d in range(depth):
        if d==0: #les images sont initialement normalisé en prétraitement donc norm=false
          x = Conv3D(Nfilter_start, kernel_size=ks, strides=1, kernel_initializer='he_normal', padding='same')(x)
          x = Conv3D(Nfilter_start, kernel_size=ks, strides=1, kernel_initializer='he_normal', padding='same')(x)
        else:
          x = encoder_step(x, Nfilter_start*np.power(2,d), ks)
        
        layers_to_concatenate.append(x)

    # bottlenek
    x = bottlenek(x, Nfilter_start*np.power(2,depth-1), ks)
    
    # decoder
    for d in range(depth-2, -1, -1): 
        x = decoder_step(x, layers_to_concatenate.pop(), Nfilter_start*np.power(2,d), ks)
    
    # classifier
    last = Conv3DTranspose(4, kernel_size=ks, strides=2, padding='same', kernel_initializer='he_normal', activation='softmax', name='output_generator')(x)
    
    del(x,layers_to_concatenate)
    return Model(inputs=inputs, outputs=last, name='Generator')


    




# *  **WT = ET ∪ ED ∪ NCR/NET**
# *  **TC = ET ∪ NCR/NET**
# *  **ET = ET**



1.   ED (edema -> classe 2)
2.   ET (enhancing_tumor -> classe 3)
3.   NCR/NET (non_enhancing_tumor -> classe 1)
4.   BG (background -> classe 0)







In [ ]:
rien=1

In [ ]:
!mkdir data

In [ ]:
t1_list = sorted(glob.glob(DATA_PATH+'/*/*_t1.nii'))
t2_list = sorted(glob.glob(DATA_PATH+'/*/*_t2.nii'))
t1ce_list = sorted(glob.glob(DATA_PATH+'/*/*_t1ce.nii'))
flair_list = sorted(glob.glob(DATA_PATH+'/*/*_flair.nii'))
# seg_list = sorted(glob.glob(DATA_PATH+'/*/*_seg.nii.gz'))

In [ ]:
#*******************MAIN**********************************************************
#*********************************************************************************
#EXTRACTION OF DATA , T1 T2 T1C TFLAIR GROUNDTRUE
BATCH_SIZE = 4
ALPHA = 5
EPOCHS = 5
NB_CLASSES = 4
N_PATCHES = 1

JUMP = False
pred_img = (PATH + f'/{VERSION}'+'_pred@epoch_{:03d}.png').format((EXEC-1)*EPOCHS) #0,8,16
if os.path.exists(pred_img)==True:
    JUMP = True


Nim = len(t1_list)
idx = np.arange(Nim)

#Generate validation set provided by BraTS challenge

sets = {'train': [], 'valid': []}

for i in idx:
    sets['valid'].append([t1_list[i], t2_list[i], t1ce_list[i], flair_list[i]])
sets['valid'].append([t1_list[0], t2_list[0], t1ce_list[0], flair_list[0]])      
del(t1_list,t2_list,t1ce_list,flair_list,idx,Nim)

# train_gen = DataGenerator(sets['train'], batch_size=BATCH_SIZE, n_classes=NB_CLASSES, n_patches=N_PATCHES, augmentation=True)
valid_gen = DataGenerator(sets['valid'], batch_size=BATCH_SIZE, n_classes=NB_CLASSES, n_patches=N_PATCHES, augmentation=True)



# Train the uGAN model

In [ ]:
h = "EVALUATION"

In [ ]:
%%capture cap_h --no-stderr
print(h)

In [ ]:
with open(PATH + f'/{VERSION}_h_{EXEC}.txt', 'w') as f:
    f.write(cap_h.stdout)

In [ ]:
#Validation 

G = Generator()


G.load_weights(INPUT_PATH  + '/brats_22_Generator.h5')


stats={'dice':[],'spec':[],'sen':[],'hau95':[]}
i=0

for Xb,Zb,AFFb in tqdm(valid_gen):
  i=i+1
  print(i)
  for k in range(len(Xb)):
    segG=np.empty((240,240,155,4))
    segG=np.zeros_like(segG)

    tempy=np.empty((160,192,128,4))
    tempy=np.zeros_like(tempy)
    part1=Xb[k,0:128,0:128,:,:]
    part2=Xb[k,0:128,64:192,:,:]
    part3=Xb[k,32:160,0:128,:,:]
    part4=Xb[k,32:160,64:192,:,:]
    Batch=np.empty((4,128,128,128,4))
    Batch[0,:]=part1
    Batch[1,:]=part2
    Batch[2,:]=part3
    Batch[3,:]=part4
    
    segBatch=G.predict(Batch)
    
    segG1 = segBatch[0,:]
    segG2 = segBatch[1,:]
    segG3 = segBatch[2,:]
    segG4 = segBatch[3,:]
    
    tempy[0:128,0:128,:,:]=segG1
    tempy[0:128,64:192,:,:]=segG2
    tempy[128:160,0:128,:,:]=segG3[96:128,0:128,:,:]
    tempy[128:160,128:192,:,:]=segG4[96:128,64:128,:,:]
   
    segG[40:200,34:226,8:136,:]=tempy

    case = Zb[k]
    case=case[21:][:15]
    
    segG_all = np.argmax(segG, axis=-1)
    segG_all[segG_all==3]=4 
    
#     segTr_all = np.argmax(segTr, axis=-1)
#    save the segmentation file
    final_img = nib.Nifti1Image(segG_all,AFFb[k])
    nib.save(final_img, os.path.join(PATHVAL,case+'.nii.gz'))
    print(segG_all.shape)
    
    del(segG,segG_all)
    



In [ ]:
! zip -r './results.zip' {PATH+'/*'}

In [ ]:
from IPython.display import FileLink
FileLink(r'./results.zip')
# then in colab : !wget "https://....kaggle.net/...../results.zip"